In [1]:
# ALPLA demo- Implementation with a small scale dataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import OneClassSVM
import joblib

# import sys
# print("Python version")
# print(sys.version)


np.set_printoptions(threshold=np.inf)

# Load measurement file:
measurement = np.load('dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']



In [2]:

# print(measurement['header'].dtype)
# print(measurement['data'].dtype)

# print()
# print('"timestamp"')
# print(f"[timestamp].shape ---> {measurement['data']['timestamp'].shape}")
# print(f"[timestamp] ---> {measurement['data']['timestamp'][1]}")
# print()

# print('"tag_position"')
# print(f"[tag_position].shape ---> {measurement['data']['tag_position'].shape}")
# print(f"[tag_position] ---> {measurement['data']['tag_position'][0]}")
# print()

# print('"speeds"')
# print(f"[speeds].shape ---> {measurement['data']['speeds'].shape}")
# print(f"[speeds] ---> {measurement['data']['speeds'][0]}")
# print()


# print('"cirs"')
# print(f"[cirs].shape ---> {measurement['data']['cirs'].shape}")
# print(f"[cirs] ---> {measurement['data']['cirs'][0]}")
# print()


# print('"rssis"')
# print(f"[rssis].shape ---> {measurement['data']['rssis'].shape}")
# print(f"[rssis] ---> {measurement['data']['rssis'][0]}")
# print()

In [36]:

# Understanding Array
# Our testing CIR dataset
simple_clone_dataset = {
    'cirs': np.array(
        [
            [ # CIR 1
                [ # Alice - channel
                    [1, 2],
                    [3, 4],
                    [5, 6] # .... [251]
                ],
                [ # other channel
                    [9, 10],
                    [11, 12],
                    [13, 14]
                ],
                [ # other channel
                    [170, 180],
                    [190, 200],
                    [210, 220]
                ]
            ],
            [ # CIR 2
                [ # Alice
                    [25, 26],
                    [27, 28],
                    [29, 30]
                ],
                [
                    [33, 34],
                    [35, 36],
                    [37, 38]
                ],
                [
                    [410, 420],
                    [430, 440],
                    [450, 460]
                ]
            ],
            [ # CIR 3
                [
                    [49, 50],
                    [51, 52],
                    [53, 54]
                ],
                [
                    [57, 58],
                    [59, 60],
                    [61, 62]
                ],
                [
                    [650, 660],
                    [670, 680],
                    [690, 700]
                ]
            ]
        ]
    )
}


# Create the structured array for header
new_header = np.array(
    [
        (
            [
                [204., 2859., 1150.],
                [5658., 2856., 1150.],
                [2900., 184., 1150.],
            ],
            [
                ('t0', 'a0', 0), #0 Bob -> Alice
                ('t0', 'a1', 0), #1 Bob -> Eve1
                ('t0', 'a2', 0), #2 Bob -> Eve2 
                ('a0', 't0', 0), #3 Alice -> Bob * (legitimate channel)
                ('a0', 'a1', 0), #4 Alice -> Eve1 
                ('a0', 'a2', 0), #5 Alice -> Eve2
                ('a1', 't0', 0), #6 Eve1 -> Bob * (illegitimate channel)
                ('a1', 'a0', 0), #7 Eve1 -> Alice
                ('a1', 'a2', 0), #8 Eve1 -> Eve2
                ('a2', 't0', 0), #9 Eve2 -> Bob * (illegitimate channel)
                ('a2', 'a0', 0), #10 Eve2 -> Alice
                ('a2', 'a1', 0), #11 Eve2 -> Eve1
                ('t0', 'a0', 1), #12 Bob -> Alice
                ('t0', 'a1', 1), #13 Bob -> Eve1
                ('t0', 'a2', 1)  #14 Bob -> Eve2
            ]
        )
    ], dtype = [
                    ('anchor_positions', '<f8', (3, 3)), 
                    (
                        'channels', 
                        [ ('transmitter', '<U2'), ('receiver', '<U2'), ('index', 'u1')], 
                        (15,)
                    )
    ]
)


print(header.dtype)
print(simple_clone_dataset['cirs'].shape)
# print(new_data['cirs'].shape)
# print(new_header_channels['channels'])

[('anchor_positions', '<f8', (3, 3)), ('channels', [('transmitter', '<U2'), ('receiver', '<U2'), ('index', 'u1')], (15,))]
(3, 3, 3, 2)


In [37]:

dataset_slice = data['cirs']
X_train = simple_clone_dataset['cirs']
print(X_train)
# Split data into training and test sets
# X_train, X_test = train_test_split(dataset_slice, test_size=0.2, random_state=42)
# print(X_train.shape)
# print(X_test.shape)

[[[[  1   2]
   [  3   4]
   [  5   6]]

  [[  9  10]
   [ 11  12]
   [ 13  14]]

  [[170 180]
   [190 200]
   [210 220]]]


 [[[ 25  26]
   [ 27  28]
   [ 29  30]]

  [[ 33  34]
   [ 35  36]
   [ 37  38]]

  [[410 420]
   [430 440]
   [450 460]]]


 [[[ 49  50]
   [ 51  52]
   [ 53  54]]

  [[ 57  58]
   [ 59  60]
   [ 61  62]]

  [[650 660]
   [670 680]
   [690 700]]]]


In [38]:
# Step 1.1 - Extract real and imaginary parts for Alice and Eve (assuming channels 0 and 1)
# channel 3 is the legitimate channel between Alice and Bob
alice_real = X_train[:3, 2, :, 0]
alice_imag = X_train[:3, 2, :, 1]


# Step 1.2 - Compute magnitudes
alice_complex = alice_real + 1j * alice_imag
alice_magnitude = np.abs(alice_complex)

Real
[[170 190 210]
 [410 430 450]
 [650 670 690]]
imag
[[180 200 220]
 [420 440 460]
 [660 680 700]]
MAgnitude
[[247.58836806 275.86228448 304.13812651]
 [586.94122363 615.22353661 643.50602173]
 [926.33687177 954.62034338 982.90386102]]


In [83]:
# Scaling Real, Imaginary and Magnitude parts
scaler = MinMaxScaler()

# Fit and transform the features
alice_real_scaled = scaler.fit_transform(alice_real)
alice_imag_scaled = scaler.fit_transform(alice_imag)
alice_mag_scaled = scaler.fit_transform(alice_magnitude)

# print("real")
# print(alice_real_scaled)

# print("imag")
# print(alice_imag_scaled)

# print("mag")
# print(alice_mag_scaled)

# feature matrix
alice_features = np.column_stack((alice_real, alice_imag, alice_magnitude))
# print('Alice feature matrix')
# print(alice_features)


In [46]:
ocsvm = OneClassSVM(kernel='linear', gamma='auto', nu=0.01)
ocsvm.fit(alice_features)

OneClassSVM(gamma='auto', kernel='linear', nu=0.01)

In [84]:
# Function to update the data vector
def update_features(features, new_cir):
    # Remove the oldest CIR (first row)
    updated_features = np.delete(features, 0, axis=0)
    # Append the new CIR to the end
    updated_features = np.vstack([updated_features, new_cir])
    return updated_features

In [85]:
# Testing and updating
# print("Before")
# print(alice_features.shape)
# print(alice_features)
# for i in range(X_test.shape[0]):
for i in range(2):
    # Extract the current test CIR
    incoming_real = X_train[i, 2, :, 0]
    incoming_imag = X_train[i, 2, :, 1]
    incoming_mag = np.abs(incoming_real + 1j * incoming_imag)


    
    test_real_scaled = scaler.transform(incoming_real.reshape(1, -1))
    test_imag_scaled = scaler.transform(incoming_imag.reshape(1, -1))
    test_mag_scaled = scaler.transform(incoming_mag.reshape(1, -1))
    
    test_features = np.column_stack((test_real_scaled, test_imag_scaled, test_mag_scaled))
    
    print('Test feature shape and values (before update)')
    print(test_features.shape)
    print(test_features)
    alice_features = update_features(alice_features, test_features)

    # # Scale the test CIR
    # test_real_scaled = scaler.transform(incoming_real.reshape(1, -1))
    # test_imag_scaled = scaler.transform(incoming_imag.reshape(1, -1))
    # test_mag_scaled = scaler.transform(incoming_mag.reshape(1, -1))
 
    # # # Create a feature vector for the test CIR
    # test_features = np.column_stack((test_real_scaled, test_imag_scaled, test_mag_scaled))

    # # # Predict using the OCC-SVM
    # prediction = ocsvm.predict(test_features)

    # # # If the CIR is accepted, update the data vector
    # if prediction == 1:
    #     print('accepted')
    #     alice_features = update_features(alice_features, test_features)
        # print(f"CIR {i} accepted, updated features:")
        # print(alice_features)


# print("After")
# print(alice_features.shape)
# print(alice_features)
# The updated alice_features will contain the most recent CIRs

Test feature shape and values (before update)
(1, 9)
[[-0.11431092 -0.1264991  -0.13869016 -0.09957793 -0.11176631 -0.12395753
   0.          0.          0.        ]]
Test feature shape and values (before update)
(1, 9)
[[0.23928102 0.22708786 0.21489281 0.25401401 0.24182065 0.22962543
  0.49996848 0.49997381 0.49997794]]
